<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-1-Introduction-to-Generative-AI-and-Agentic-AI/M1LC3_Conditional_Routing_in_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional Routing in LangGraph

In LangGraph, conditional routing allows the dynamic determination of the next node(s) to execute based on the current state.

This is achieved using conditional edges, which evaluate a function using certain defined logic, to decide the subsequent path in the graph.

This mechanism enables the creation of adaptable workflows that respond to varying conditions during execution. Let's dive deep into conditional routing in this demo.

![](https://i.imgur.com/uX12AyO.png)

## Install Dependencies

In [ ]:
!pip install langchain==0.3.27 langchain-openai==0.3.29 langgraph==0.6.5 --quiet

## Creating the Graph with Conditional Edges

[Edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#edges) connect the nodes.

Normal Edges are used if you want to *always* go from, for example, `node_1` to `node_2`.

[Conditional Edges](https://langchain-ai.github.io/langgraph/reference/graphs/?h=conditional+edge#langgraph.graph.StateGraph.add_conditional_edges) are used want to *optionally* route between nodes.

Conditional edges are implemented as functions that return the next node to visit based upon some logic.

![](https://i.imgur.com/uX12AyO.png)

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# Define the state structure
class State(TypedDict):
    input: str
    weather: str

# Initialize the StateGraph
graph_builder = StateGraph(State)

# Define node functions
def weather_node(state: State) -> State:
    print("\nChecking the weather...")
    # here you might want to call some tool or logic and get the weather
    # for now we will just pass the state along which has the weather key - value pair
    print('Weather entered by user is:', state['input'])
    return state  # Just pass the state along

def rainy_node(state: State) -> State:
    print("It's rainy. Take an umbrella!")
    return {'weather': 'rainy confirmed'}

def sunny_node(state: State) -> State:
    print("It's sunny. Wear sunglasses!")
    return {'weather': 'sunny confirmed'}

# Define the routing function
def weather_routing_fn(state: State) -> str:
    user_input = state['input']
    if user_input.lower() == 'rainy':
        return "goto_rainy_node" # name of the rainy node we will define shortly
    elif user_input.lower() == 'sunny':
        return "goto_sunny_node" # name of the sunny node we will define shortly
    else:
        raise ValueError("Invalid weather condition!")

# Add nodes to the graph
graph_builder.add_node("weather_node", weather_node)
graph_builder.add_node("rainy_node", rainy_node)
graph_builder.add_node("sunny_node", sunny_node)

# Define edges
graph_builder.add_edge(START, "weather_node")
graph_builder.add_conditional_edges("weather_node",
                                    # the routing function will route to rainy_node if weather state is 'rainy'
                                    # the routing function will route to sunny_node if the weather state is 'sunny'
                                    weather_routing_fn, # routing function
                                    {
                                        "goto_rainy_node": "rainy_node",
                                        "goto_sunny_node": "sunny_node"
                                    } # possible nodes where the routing function can end up going
                                   )
graph_builder.add_edge("rainy_node", END)
graph_builder.add_edge("sunny_node", END)

# Compile the graph
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Execute the graph with an initial state for "rainy"
initial_state_rainy = {"input": "rainy"}
result_rainy = graph.invoke(initial_state_rainy)

In [ ]:
result_rainy

In [ ]:
# Execute the graph with an initial state for "sunny"
initial_state_sunny = {"input": "sunny"}
result_sunny = graph.invoke(initial_state_sunny)

In [ ]:
# Execute the graph with an initial state for "sunny"
initial_state_sunny = {"input": "humid"}
result_sunny = graph.invoke(initial_state_sunny)

## Conditional Routing with More Nodes

The more nodes you have for custom logic you just need to define the node functions and then put in relevant conditions for the function which would do the conditional routing as shown in the example below.

![](https://i.imgur.com/FbiQBYZ.png)

In [ ]:
# Define the state structure
class State(TypedDict):
    input: str
    weather: str

# Initialize the StateGraph
graph_builder = StateGraph(State)

# Define node functions
def weather_node(state: State) -> State:
    print("\nChecking the weather...")
    # here you might want to call some tool or logic and get the weather
    # for now we will just pass the state along which has the weather key - value pair
    print('Weather entered by user is:', state['input'])
    return state  # Just pass the state along

def rainy_node(state: State) -> State:
    print("It's rainy. Take an umbrella!")
    return {'weather': 'rainy confirmed'}

def sunny_node(state: State) -> State:
    print("It's sunny. Wear sunglasses!")
    return {'weather': 'sunny confirmed'}

def other_node(state: State) -> State:
    print("Sorry I can't detect the weather yet!")
    return {'weather': 'not confirmed'}

# Define the routing function
def weather_routing_fn(state: State) -> str:
    user_input = state['input']
    if user_input.lower() == 'rainy':
        return "goto_rainy_node" # name of the rainy node we will define shortly
    elif user_input.lower() == 'sunny':
        return "goto_sunny_node" # name of the sunny node we will define shortly
    else:
        return "goto_other_node"

# Add nodes to the graph
graph_builder.add_node("weather_node", weather_node)
graph_builder.add_node("rainy_node", rainy_node)
graph_builder.add_node("sunny_node", sunny_node)
graph_builder.add_node("other_node", other_node)

# Define edges
graph_builder.add_edge(START, "weather_node")
graph_builder.add_conditional_edges("weather_node",
                                    # the routing function will route to rainy_node if weather state is 'rainy'
                                    # the routing function will route to sunny_node if the weather state is 'sunny'
                                    weather_routing_fn, # routing function
                                    {
                                        "goto_rainy_node": "rainy_node",
                                        "goto_sunny_node": "sunny_node",
                                        "goto_other_node": "other_node"
                                    } # possible nodes where the routing function can end up going
                                   )
graph_builder.add_edge("rainy_node", END)
graph_builder.add_edge("sunny_node", END)
graph_builder.add_edge("other_node", END)

# Compile the graph
graph = graph_builder.compile()

In [ ]:
# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"input": "rainy"})

In [ ]:
result

In [ ]:
result = graph.invoke({"input": "Sunny"})
result

In [ ]:
result = graph.invoke({"input": "Stormy"})
result